In [1]:
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import QFileDialog, QInputDialog, QLineEdit, QMessageBox
import xml.etree.ElementTree as ET
import sys
from testallproject import Ui_MainWindow


file_opened = False
parser = ET.XMLParser(encoding="utf-8")
#tree_char = ET.parse('ShablonPawn.xml', parser=parser)
#root_char = tree_char.getroot()
rel = []
input_all = []
arr_all_id_chars = []
input_dialog = []
input_answer = []
input_str_ans = []
quest_person = "true"
quest_robot = "true"
gender = "М"
flag_answer = False

class Xml_perevod(QtWidgets.QMainWindow):

    def __init__(self):
        super(Xml_perevod, self).__init__()
        self.base = Ui_MainWindow()
        self.base.setupUi(self)
        self.base.name_ext.setVisible(False)
        self.base.exit.setVisible(False)
        self.visibleAnswer(False)
        self.base.open_file_dialog.setVisible(False)
        self.base.save_all.clicked.connect(self.enterHuman)
        self.base.save_rel.clicked.connect(self.enterRel)
        self.base.comboGenger.activated[str].connect(self.onChangedGender)
        self.base.comboQuest.activated[str].connect(self.onChangedQuest_Man)
        self.base.save_all_robot.clicked.connect(self.enterRobot)
        self.base.comboQuest_robot.activated[str].connect(self.onChangedQuest_robot)
        self.base.clean_button.clicked.connect(self.clearAll)
        self.base.clean_button_robot.clicked.connect(self.clearAll)
        self.base.open_file.clicked.connect(self.openFile)
        self.base.delete_char.clicked.connect(self.removeRecordChar)
        self.base.create_file_dialog.clicked.connect(self.createXml)
        self.base.answers_button.clicked.connect(self.haveAnswer)
        self.base.save_answer.clicked.connect(self.save_one_answer)
        self.base.save_all_dialog.clicked.connect(self.enter_full_dialog)

    def save_one_answer(self):
        input_answer.append(self.base.tonode.text())
        input_answer.append(self.base.actquestA.text())
        input_answer.append(self.base.numquestA.text())
        input_answer.append(self.base.posofquestA.text())
        input_ans = self.base.answer.text()
        answerstr = "<answer> <text>" + input_ans + "</text></answer> "
        input_str_ans.append(answerstr)
        print(len(input_str_ans) + 1)
        self.base.answer_label.setText("Ответ № " + str(len(input_str_ans) + 1))
        self.base.answer.clear()


    def enter_full_dialog(self):
        x = 0
        input_dialog.append(self.base.tochar.text())
        input_dialog.append(self.base.actquest.text())
        input_dialog.append(self.base.numquest.text())
        input_dialog.append(self.base.posofquest.text())
        input_node = self.base.node.text()
        nodestr = "<node>" + "<npctext>" + input_node + "</npctext>"
        if flag_answer ==True:
            nodestr +="<answers>"
            for i in range(len(input_str_ans)):
                nodestr += input_str_ans[i]
            nodestr += "</answers> </node>"
        else:
            nodestr += "</node>"
        print(nodestr)
        str_add = ET.fromstring(nodestr)
        for elem in str_add.iter('node'):
            elem.set('tochar', input_dialog[0])
            elem.set('actquest', input_dialog[1])
            elem.set('numquest', input_dialog[2])
            elem.set('posofquest', input_dialog[3])
        if flag_answer == True:
            for elem in str_add.iter('answer'):
                print(x)
                elem.set('tonode', input_answer[x])
                elem.set('actquestA', input_answer[x+1])
                elem.set('numquestA', input_answer[x+2])
                elem.set('posofquestA', input_answer[x+3])
                x += 4

        self.base.answers_button.setEnabled(True)
        self.visibleAnswer(False)
        root_dialog.append(str_add)
        ET.indent(tree_dialog, space='  ', level=0)
        tree_dialog.write('output.xml', encoding="utf-8")


    def visibleAnswer(self, flag):
        self.base.answer_label.setVisible(flag)
        self.base.answer.setVisible(flag)
        self.base.save_answer.setVisible(flag)
        self.base.tonode_label_2.setVisible(flag)
        self.base.tonode.setVisible(flag)
        self.base.actquestA_label.setVisible(flag)
        self.base.actquestA.setVisible(flag)
        self.base.numquestA_label.setVisible(flag)
        self.base.numquestA.setVisible(flag)
        self.base.posofquestA_label.setVisible(flag)
        self.base.posofquestA.setVisible(flag)

    def haveAnswer(self):
        question = QMessageBox.question(self, "Ответы", "Будет ли диалог повторяться в игре?",
                                        QMessageBox.Yes | QMessageBox.No)
        if question == QMessageBox.Yes:
            self.visibleAnswer(True)
            global flag_answer
            flag_answer = True
        self.base.answers_button.setEnabled(False)



    def createXml(self):
        global root_dialog
        global tree_dialog
        root_dialog = ET.Element("dialogue")

        question = QMessageBox.question(self, "Повторение", "Будет ли диалог повторяться в игре?",
                                        QMessageBox.Yes | QMessageBox.No)
        if question == QMessageBox.Yes:
            root_dialog.set("more", str(1))
        else:
            root_dialog.set("more", str(0))

        tree_dialog = ET.ElementTree(root_dialog)
        ET.indent(tree_dialog, space='  ', level=0)
        with open('output.xml', 'wb') as fh:
            tree_dialog.write(fh)

    def removeRecordChar(self):
        current_index = self.base.listWidget_Char.currentRow()
        print(current_index)
        item = self.base.listWidget_Char.item(current_index)
        if item is None:
            return
        flag = False



        question = QMessageBox.question(self, "Удалить персонажа", "Точно удалить персонажа?" + item.text(), QMessageBox.Yes | QMessageBox.No)
        if question == QMessageBox.Yes:
            for char in root_char.findall('Char'):
                ID = -100000
                if char.get('charID') != '':
                    ID = int(char.get('charID'))
                if ID == int(arr_all_id_chars[current_index]):
                    flag = True
                    root_char.remove(char)

            if flag == False:
                for char in root_char.findall('SyntheticChar'):
                    ID = -100000
                    if char.get('syntID') != '':
                        ID = int(char.get('syntID'))
                    if ID == int(arr_all_id_chars[current_index]):
                        root_char.remove(char)
            arr_all_id_chars.pop(current_index)
            removed = self.base.listWidget_Char.takeItem(self.base.listWidget_Char.currentRow())
            ET.indent(tree_char, space='  ', level=0)
            tree_char.write(res[0], encoding="utf-8")

    def openFile(self):
        global res
        res = QFileDialog.getOpenFileName(self, 'Open File')
        print(res)
        global tree_char
        global root_char
        if res[0] == '':
            print("AAAAAAAAAAAAA")
            msg = QMessageBox()
            msg.setText("Выбери файл")
            msg.exec_()
            return
        tree_char = ET.parse(res[0], parser=parser)
        root_char = tree_char.getroot()
        output = ""
        for char in root_char.findall('Char'):
            output += "Чел [Id: " + char.get('charID')
            output += " Возраст: " + char.get('age')
            output += " Пол: " + char.find('Gender').text
            output += " Кличка: " + char.find('NickName').text
            output += " Фракция: " + char.find('Fraction').text
            self.base.listWidget_Char.addItems([output])
            output = ""
            arr_all_id_chars.append(int(char.get('charID')))
        for char in root_char.findall('SyntheticChar'):
            output += "Cинтет [Id: " + char.get('syntID')
            output += " Кличка: " + char.find('NickName').text
            output += " Фракция: " + char.find('Fraction').text
            output += " Модель: " + char.find('Model').text
            self.base.listWidget_Char.addItems([output])
            output = ""
            arr_all_id_chars.append(int(char.get('syntID')))
        global file_opened
        file_opened = True
        print(res)

    def onChangedGender(self, text):
        global gender
        gender = text

    def onChangedQuest_Man(self, text):
        global quest_person
        quest_person = text

    def onChangedQuest_robot(self, text):
        global quest_robot
        quest_robot = text

    def enterRel(self):
        rel.append(self.base.rel.text())
        self.base.rel.clear()

    def enterRobot(self):
        if file_opened == False:
            print("AAAAAAAAAAAAA")
            msg = QMessageBox()
            msg.setText("Выбери файл")
            msg.exec_()
            return
        input_id = self.base.id_robot.text()
        input_all.append(self.base.model_robot.text())
        input_all.append(self.base.fraction_robot.text())
        input_all.append(self.base.nikname_robot.text())
        input_all.append(self.base.stats_robot.text())
        input_all.append(quest_robot)
        self.cleatNotAll()
        arr_all_id_chars.append(input_id)
        what_robot = ["Model", "Fraction", "NickName", "Stats", "QuestChar"]
        output = "Cинтет [Id: " + input_id + " Кличка: " + input_all[2] + " Фракция: " + input_all[1] + " Модель: " + input_all[0] + "]"
        strtest = "<SyntheticChar>"
        for i in range(len(what_robot)):
            strtest += "<" + what_robot[i] + ">" + input_all[i] + "</" + what_robot[i] + ">"
        strtest += "</SyntheticChar>"
        self.base.listWidget_Char.addItems([output])
        str_add = ET.fromstring(strtest)
        for elem in str_add.iter('SyntheticChar'):
            elem.set('syntID', input_id)
        root_char.append(str_add)
        ET.indent(tree_char, space='  ', level=0)
        tree_char.write(res[0], encoding="utf-8")
        input_all.clear()


    def enterHuman(self):
        if file_opened == False:
            print("AAAAAAAAAAAAA")
            msg = QMessageBox()
            msg.setText("Выбери файл")
            msg.exec_()
            return
        input_age = self.base.age.text()
        input_id = self.base.id.text()
        input_all.append(gender)
        input_all.append(self.base.race.text())
        input_all.append(self.base.nationality.text())
        input_all.append(self.base.fraction.text())
        input_all.append(self.base.stats.text())
        input_all.append(self.base.surname.text())
        input_all.append(self.base.name.text())
        input_all.append(self.base.nikname.text())
        input_all.append("")
        input_all.append(self.base.adress.text())
        input_all.append(quest_person)
        self.cleatNotAll()
        arr_all_id_chars.append(input_id)
        what = ["Gender", "Race", "Nationality", "Fraction", "Stats", "SurName", "Name", "NickName",
                "Relatives", "HomeLocation", "QuestChar", ]
        output = "Чел [Id: " + input_id + " Возраст: " + input_age + " Пол: " + gender + " Кличка: " + input_all[7] + " Фракция: " + input_all[3] + "]"
        strtest = "<Char>"
        for i in range(len(what)):
            if what[i] != 'Relatives':
                strtest += "<" + what[i] + ">" + input_all[i] + "</" + what[i] + ">"
            else:
                strtest += "<" + what[i] + ">"
                for j in range(len(rel)):
                    strtest += "<rel>" + rel[j] + "</rel>"

                strtest += "</" + what[i] + "> "
        strtest += "</Char>"
        self.base.listWidget_Char.addItems([output])
        rel.clear()
        str_add = ET.fromstring(strtest)
        for elem in str_add.iter('Char'):
            elem.set('charID', input_id)
            elem.set('age', input_age)
        root_char.append(str_add)
        ET.indent(tree_char, space='  ', level=0)
        tree_char.write(res[0], encoding="utf-8")
        input_all.clear()
        #self.base.save_all.setVisible(False)


    def cleatNotAll(self):
        self.base.age.clear()
        self.base.id.clear()
        self.base.surname.clear()
        self.base.name.clear()
        self.base.nikname.clear()
        self.base.adress.clear()
        self.base.nikname_robot.clear()
        self.base.stats_robot.clear()
        self.base.stats.clear()

    def clearAll(self):
        self.base.age.clear()
        self.base.id.clear()
        self.base.nationality.clear()
        self.base.fraction.clear()
        self.base.stats.clear()
        self.base.surname.clear()
        self.base.name.clear()
        self.base.nikname.clear()
        self.base.race.clear()
        self.base.adress.clear()
        self.base.model_robot.clear()
        self.base.fraction_robot.clear()
        self.base.nikname_robot.clear()
        self.base.stats_robot.clear()





app = QtWidgets.QApplication(sys.argv)
application = Xml_perevod()
application.show()
sys.exit(app.exec_())


('C:/Users/user/PycharmProjects/pythonProject2/ShablonPawn.xml', 'All Files (*)')


ValueError: invalid literal for int() with base 10: ''

SystemExit: 0

C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
